# **Konfio Take Home Test**

The dataset attached (‘dataset.zip’) consists on two csv files: ‘users.csv’ containing the
income, outcome and a label for 1000 users, and ‘credit_reports.csv’ containing all the
accounts from the user’s credit reports . (Check the appendix A and B for the 1
codebooks).
Each user is labeled as 1 if the user was a good client and 0 if the user was a bad client

1) Pick the best clients you will give a loan to, based on the model you created. It
could be as complex as you decide (even as simpler as knock out rules), as long as
the metrics support it

2) Propose an amount to be lended to those clients and a term in which the loan will
need to be paid back.

3) Finally choose an anual interest rate the lended amount must have in order to be
profitable.

## 1) Business Understanding:

As almost every business, the main objective of commercial banks is to make money, how do they do this?
Commercial banks use customer deposits to lend money to other individuals at a given interest rate. This means that for the bank to be profitable the bank needs that:
- The interest rate interest rate charged for lending money (placement rate) needs to be higher than the interest rates of deposits (captive rate) (and higher than the inflation rate).
- The borrowers pay all of its debt with its corresponding fees and interest.

This means that the bank is interested in having models which:
- Predicts with confidence when a person is not likely to repay a loan
- Select users who are more likely to pay in order to offer them a loan

For the first model the plan is to build a dataset and a label that exposes when a credit has 3 or more delayed payments and learn train a model which learns the patterns which cause this.
This model will allow us to pick the best combinations of terms and amount to be lended by trying different combinations and choosing the one with the least probability of default.

For the second model I tried 2 different approaches:
- The first one involved doing a weighted average based on the features which I considered necessary for a client to be good (and weighted them accordingly) and ranked the users based on such formula without taking into account the label provided. Before doing so I applied a few transformations like taking the log and scaling the values between 0 and 1 for equal weight across features.
- The second one involved a more common approach for the industry (but personally never heard of it before) and was to train a Logistic Regression replacing variables by its WoE (weight of evidence) and filtering them by their IV (information value). Afterwords, using a specific formula based on the regression coefficient and the features's WoE a scorecard was built in order to rank users. (https://weclouddata.com/student-project-credit-scoring-using-machine-learning/)

The reason for this is mainly due to the fact that I observed that there were users who were labeled as good but had a really questionable credit history and fitting a model to an already labeled user felt odd.

To choose the annual interest interest in order to be profitable I used the model which predicts infractors. The key of the answers lies in "profitable". As mentioned before, assuming total certainty in payments, a commercial bank is profitable if it has a rate diferential between its active and passive interest rates but the fact that in the real world uncertainty exists (in the form of default), I considered that profitability should also account for this issue. This is the reason why I decided to model the "profitable" interest rate as $$i_t = p_t + \left(\frac{1}{P(Y=0|X)}\right)^2$$ where $p_t$ is the passive interest rate, $\left(\frac{1}{P(Y=0|X)}\right)^2$ is the risk premium and $P(Y=0|X)$ is the probabily of NOT being an infractor (having 3 or more payments due).

I searched for values to use for $p_t$ and found out that in Mexico the current it has an annual rate that bank's pay for deposits is aproximately 3%. Also I searched for loans interest rates in Mexico and found out that [BBVA offers an annual interest rate between 25,75% and 45,75%](https://www.bbva.mx/personas/productos/creditos/prestamos-personales/prestamo-personal-inmediato.html#:~:text=25.75%25%20hasta%2045.75%25%20sin%20IVA,de%20tu%20evaluaci%C3%B3n%20de%20cr%C3%A9dito.) for loans between 3k and 750k so my formula was adjusted in order to at least try to yield similar results.

In order to do so, the interest rate could also be defined as $$i_t = i^B_t + \left(\frac{1}{P(Y=0|X)}\right)$$ where $i^B_t$ is the minimum interest rate determined by another bank

*Note:* From now on, the comments are in Spanish

## 2) Data Understanding:

In [ ]:
!pip3 install pandas
!pip3 install numpy
!pip3 install seaborn
!pip3 install matplotlib
!pip3 install scikit-learn
!pip3 install lightgbm
!pip3 install lightgbm
!pip3 install pandas-profiling
!pip2 install nb_black

In [ ]:
%load_ext lab_black

In [ ]:
import json
import logging
import os
from datetime import datetime, date, timedelta
import unidecode
import random

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import (
    train_test_split,
    KFold,
    cross_val_score,
    GridSearchCV,
)
from sklearn.preprocessing import (
    OneHotEncoder,
    LabelEncoder,
    PolynomialFeatures,
    StandardScaler,
    MinMaxScaler,
)
from sklearn.metrics import classification_report, plot_confusion_matrix, plot_roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.impute import SimpleImputer
from lightgbm import LGBMClassifier, LGBMRegressor
from pandas_profiling import ProfileReport

In [ ]:
random.seed(10)

In [ ]:
pd.set_option("display.float_format", lambda x: "%.3f" % x)

In [ ]:
users = pd.read_csv("users.csv")
credit_reports = pd.read_csv("credit_reports.csv")

In [ ]:
users_report = ProfileReport(users)
credit_reports_report = ProfileReport(credit_reports)

In [ ]:
users_report

In [ ]:
credit_reports_report

Tipos de cuenta:
- Revolvente: https://economipedia.com/definiciones/credito-revolving.html
- Prendario: https://economipedia.com/definiciones/credito-pignoraticio.html
- Hipoteca: https://economipedia.com/definiciones/hipoteca.html
- Quirografiario: https://economipedia.com/definiciones/prestamo-quirografario.html

Tipos de credito:
    
- Linea de Credito: https://economipedia.com/definiciones/linea-de-credito.html
- Prestamo personal: https://economipedia.com/definiciones/credito-personal.html
- Tarjeta de Credito: https://economipedia.com/definiciones/tarjeta-de-credito.html
- Credito fiscal: https://economipedia.com/definiciones/incentivo-fiscal.html

Observemos que el dataset tiene informacion nula

In [ ]:
credit_reports.isna().sum()

- Observemos que pasa con `total_credit_payments`

In [ ]:
credit_reports.query("account_type == 'Revolvente' ")[
    "total_credit_payments"
].describe()

In [ ]:
credit_reports[credit_reports["total_credit_payments"].isna()][
    ["account_type", "credit_type", "total_credit_payments", "account_closing_date"]
]

Podemos ver que son todos creditos revolventes / sin limite preestablecido que usualmente no tienen pagos preacordados

Veamos la distribucion de la variable

In [ ]:
credit_reports.query("account_type == 'Revolvente'")["total_credit_payments"].describe()

Dado que no tienen fecha de fin (intuyo que son lineas de credito que siguen abiertas al momento que se creo el dataset) y que la distribucion señala que la gran mayoria de estos creditos tienen cantidad de pagos = a 0, voy a completar los valores faltantes con dicho valor (la moda)

In [ ]:
credit_reports["total_credit_payments"] = credit_reports[
    "total_credit_payments"
].fillna(0)

- Observemos `payment_frequency`

In [ ]:
credit_reports[credit_reports["payment_frequency"].isna()]

In [ ]:
credit_reports["payment_frequency"].value_counts()

In [ ]:
credit_reports.query("user_id == 742")

In [ ]:
credit_reports.query(
    "institution == 'COOPERATIVA' and account_type == 'Pagos Fijos' and credit_type == 'Préstamo Personal '"
)["payment_frequency"].value_counts()

Podemos ver que la forma de pago mas usual para dicha institucion, linea de credito y tipo de cuenta es mensual. Ademas, el usuario opto siempre por pagos mensuales.
De todas maneras dado que tenemos un closing_date de aproximadamente 6 meses de diferencia (tiene sentido que sean 2 pagos trimestrales), voy a optar por eliminar la observacion.

In [ ]:
credit_reports = credit_reports[
    ~credit_reports["payment_frequency"].isna()
].reset_index(drop=True)

- Observemos `amount_to_pay_next_payment`

In [ ]:
credit_reports[credit_reports["amount_to_pay_next_payment"].isna()]

Segun entiendo `amount_to_pay_next_payment` va a estar influenciada por `maximum credit limit` y `total_credit_payments`, es decir:

- `amount_to_pay_next_payment` = `maximum_credit_amount` * (1 + i) / `total_credit_payments` donde i puede ser una tasa catorcenal, mensual, quincenal, etc

Pareceria ser posible sacar la tasa de interes de este modo para el caso de prestamos personales con pagos fijos que no hayan tenido ningun tipo de deuda

De la formula anterior podemos saber que:
    
- i_t = ((`amount_to_pay_next_payment` * `total_credit_payments`) / `maximum_credit_amount`) - 1

(De todas maneras no voy a usar este dato porque los resultados no tenian sentido (anexo)

Ante la duda voy a optar por eliminar estas observaciones que contienen datos faltantes que parecen irrecuperables

In [ ]:
credit_reports = credit_reports[
    ~credit_reports["amount_to_pay_next_payment"].isna()
].reset_index(drop=True)

Obervamos: `maximum_credit_amount`

In [ ]:
credit_reports[~credit_reports["maximum_credit_amount"].isna()]

Podria recuperar el valor asumiendo que maximum_credit_amount == amount_to_pay_next_payment, ante la duda, prefiero eliminar la observacion

In [ ]:
credit_reports = credit_reports[
    ~credit_reports["maximum_credit_amount"].isna()
].reset_index(drop=True)

Observemos: `credit_limit`

Algo que me llama la atencion de esta variable es que esta variable el anexo la define como: 
>Credit limit for this account

Pero despues tenemos la variable `maximum_credit_amount` que significa
> Maximum amount of credit used by the consumer.

Asi que en teoria maximum_credit_amount <= credit_limit, validemoslo

In [ ]:
credit_reports.query("maximum_credit_amount <= credit_limit").shape[
    0
] / credit_reports.shape[0]

In [ ]:
np.log(credit_reports.credit_limit + 1).describe()

In [ ]:
credit_reports.credit_limit.value_counts(normalize=True)

Solo el 42% de los datos cumplem con esa hipotesis, posiblemente porque 43% de las observaciones tienen credit_limit == 0.

In [ ]:
credit_reports.query(
    "account_type == 'Pagos Fijos' and credit_type == 'Préstamo Personal ' and credit_limit != 0"
)

Pareceria ser que en los casos en los que no es 0 el credit_limit es igual al maximum_credit_amount. Tiene sentido dado que en un Prestamo Personal no exite tal cosa como un credito limite sino que el monto a pagar esta establecido previamente.

In [ ]:
credit_reports.corr()["credit_limit"]

In [ ]:
credit_reports.query(
    "account_type == 'Pagos Fijos' and credit_type == 'Préstamo Personal ' and credit_limit != 0"
).apply(lambda x: x.maximum_credit_amount == x.credit_limit, axis=1).mean()

El 93% de las veces se valida esta informacion.

De todas maneras antes la diversidad de tipos de credito prefiero directamente eliminar las observaciones con datos faltantes

In [ ]:
credit_reports.query(
    "account_type == 'Pagos Fijos' and credit_type == 'Préstamo Personal '"
)[["maximum_credit_amount", "credit_limit"]]

In [ ]:
credit_reports = credit_reports[~credit_reports["credit_limit"].isna()].reset_index(
    drop=True
)

In [ ]:
credit_reports.isna().sum()

In [ ]:
credit_reports = credit_reports[
    ~credit_reports["payment_frequency"].isna()
].reset_index(drop=True)

In [ ]:
credit_reports.query("current_balance == 0 and number_of_payments_due > 0").shape

No entiendo muy bien el funcionamiento de current_balance, figura que todo esta pago cuando tiene deudas pendientes, voy a optar por eliminarla

## 3) Data Preparation

In [ ]:
credit_reports.query("total_credit_payments == 0")["account_type"].value_counts()

Dado que la consigna consulta sobre prestamos y para simplificar el analisis, voy a tomar solo el historial crediticio para Prestamos Personales con Pagos Fijos que hayan sido utilizados.
Ademas, voy a filtrar por las observaciones que tengan total_credit_payments > a 0 dado que no le encuentro sentido a que sea == a 0.

In [ ]:
credit_reports = credit_reports.query(
    "credit_type == 'Préstamo Personal ' and total_credit_payments != 0 and maximum_credit_amount > 0"
).reset_index(drop=True)

In [ ]:
credit_reports["account_opening_date"] = pd.to_datetime(
    credit_reports.account_opening_date
)
credit_reports["account_closing_date"] = pd.to_datetime(
    credit_reports["account_closing_date"]
)

In [ ]:
credit_reports["payments_due_ratio"] = (
    credit_reports["number_of_payments_due"] / credit_reports["total_credit_payments"]
)

In [ ]:
credit_reports["loan_amount"] = np.where(
    credit_reports["credit_limit"] > 0,
    credit_reports["credit_limit"],
    credit_reports["maximum_credit_amount"],
)

In [ ]:
credit_reports["worst_delinquent_payments"] = np.where(
    credit_reports["worst_delinquency"] > credit_reports["number_of_payments_due"],
    credit_reports["worst_delinquency"],
    credit_reports["number_of_payments_due"],
)

In [ ]:
credit_reports["infractor"] = np.where(
    credit_reports["worst_delinquent_payments"] > 3, 1, 0
)

In [ ]:
mapping_frequency = {
    "mensual": 30,
    "semanal": 7,
    "una sola exhibicion": 1,
    "quincenal": 15,
    "catorcenal": 14,
    "anual": 360,
    "trimestral": 90,
    "bimestral": 60,
}

In [ ]:
credit_reports["payment_frequency_delta"] = credit_reports["payment_frequency"].apply(
    lambda x: timedelta(mapping_frequency.get(unidecode.unidecode(x.lower())))
)

credit_reports["payment_frequency_days"] = credit_reports[
    "payment_frequency_delta"
].dt.days

In [ ]:
credit_reports["expected_end"] = credit_reports["account_opening_date"] + (
    credit_reports["total_credit_payments"] * credit_reports["payment_frequency_delta"]
)

In [ ]:
credit_reports["pre_cancelled"] = (
    credit_reports["account_closing_date"] < credit_reports["expected_end"]
).astype(int)

In [ ]:
credit_reports["duration_days"] = (
    credit_reports["expected_end"] - credit_reports["account_opening_date"]
).dt.days

### Users features

In [ ]:
users = users.rename(columns={"id": "user_id"})
users = users.rename(columns={"class": "is_good"})

In [ ]:
full_credit_reports = pd.merge(users, credit_reports, on="user_id")

In [ ]:
users.shape

In [ ]:
credit_reports.shape

In [ ]:
full_credit_reports.shape

In [ ]:
users_features = users.copy()

In [ ]:
features_map = {
    #     "min": ["worst_delinquency_past_due_balance"],
    #     "max": ["loan_amount", "worst_delinquent_payments"],
    "mean": [
        "loan_amount",
        "worst_delinquent_payments",
        "infractor",
        "payment_frequency_days",
        "pre_cancelled",
        "duration_days",
    ],
    "count": ["loan_amount"],
}

In [ ]:
for agg, features in features_map.items():
    for feature in features:
        if agg == "min":
            users_features[f"{agg}_{feature}"] = full_credit_reports.groupby("user_id")[
                feature
            ].min()
        elif agg == "mean":
            users_features[f"{agg}_{feature}"] = full_credit_reports.groupby("user_id")[
                feature
            ].mean()
        elif agg == "max":
            users_features[f"{agg}_{feature}"] = full_credit_reports.groupby("user_id")[
                feature
            ].max()
        elif agg == "count":
            users_features[f"{agg}_{feature}"] = full_credit_reports.groupby("user_id")[
                feature
            ].count()

In [ ]:
users_features["monthly_net_income"] = (
    users_features["monthly_income"] - users_features["monthly_outcome"]
)
users_features = users_features.drop(
    ["user_id", "monthly_income", "monthly_outcome"], axis=1
)
users_features = users_features.dropna()  # Drop users without credit records

In [ ]:
users_features.query("is_good == 1").sort_values(
    ["mean_worst_delinquent_payments", "mean_infractor"], ascending=False
)[:5]

Me parece raro que se cataloguen como buenos usuarios que tienen un histora crediticio tan malo. Concentremesnos en el usuario 400

In [ ]:
full_credit_reports.query("user_id == 400")[
    [
        "user_id",
        "maximum_credit_amount",
        "number_of_payments_due",
        "worst_delinquency",
        "worst_delinquency_past_due_balance",
    ]
]

El usuario incumplio pagos en sus 3 creditos por casi la totalidad de lo usado.

In [ ]:
X_users = users_features.drop("is_good", axis=1)
y_users = users_features["is_good"]

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(
    X_users.corr(),
    xticklabels=X_users.corr().columns,
    yticklabels=X_users.corr().columns,
    center=0,
    cmap=sns.diverging_palette(220, 20, as_cmap=True),
    annot=True,
)

### Infractor features

Creo features historicas laggeadas y elimino features inexistentes al momento de evaluar para evitar cualquier tipo de data leakage.

In [ ]:
infractor_features = full_credit_reports.copy()

In [ ]:
infractor_features = infractor_features.sort_values(
    ["user_id", "account_opening_date"]
).reset_index(
    drop=True
)  # Ordeno por user y fecha

In [ ]:
infractor_features["historic_total_loans_granted"] = (
    infractor_features.groupby(["user_id"])["loan_amount"]
    .apply(lambda x: x.shift().expanding().count())
    .fillna(0)
)

In [ ]:
infractor_features["historic_worst_delinquent_payments"] = (
    infractor_features.groupby(["user_id"])["worst_delinquent_payments"]
    .apply(lambda x: x.shift().expanding().max())
    .fillna(0)
)

In [ ]:
infractor_features["historic_infractor_rate"] = (
    infractor_features.groupby(["user_id"])["infractor"]
    .apply(lambda x: x.shift().expanding().mean())
    .fillna(0)
)

In [ ]:
infractor_features["historic_pre_cancelled_rate"] = (
    infractor_features.groupby(["user_id"])["pre_cancelled"]
    .apply(lambda x: x.shift().expanding().mean())
    .fillna(0)
)

In [ ]:
infractor_features["loan_created_year"] = infractor_features[
    "account_opening_date"
].dt.year.astype(str)
infractor_features["loan_created_month"] = infractor_features[
    "account_opening_date"
].dt.month.astype(str)
infractor_features["loan_created_weekday"] = infractor_features[
    "account_opening_date"
].dt.weekday.astype(str)
infractor_features["loan_expected_end_year"] = infractor_features[
    "expected_end"
].dt.year.astype(str)
infractor_features["loan_expected_end_month"] = infractor_features[
    "expected_end"
].dt.month.astype(str)
infractor_features["loan_expected_end_weekday"] = infractor_features[
    "expected_end"
].dt.weekday.astype(str)

In [ ]:
infractor_features = infractor_features.drop(
    [
        "current_balance",
        "past_due_balance",
        "worst_delinquency_date",
        "payment_frequency",
        "payment_frequency_delta",
        "worst_delinquency",
        "worst_delinquent_payments",
        "expected_end",
        "account_opening_date",
        "account_closing_date",
        "pre_cancelled",
        "payments_due_ratio",
        "number_of_payments_due",
        "worst_delinquency_past_due_balance",
        "amount_to_pay_next_payment",
        "maximum_credit_amount",
        "credit_limit",
        "is_good",
        "account_type",
        "credit_type",
    ],
    axis=1,
)

In [ ]:
X_infractor = infractor_features.drop(["infractor", "user_id"], axis=1)
y_infractor = infractor_features["infractor"]

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(
    X_infractor.corr(),
    xticklabels=X_infractor.corr().columns,
    yticklabels=X_infractor.corr().columns,
    center=0,
    cmap=sns.diverging_palette(220, 20, as_cmap=True),
    annot=True,
)

# 4) Modelling

### Scoring Model

In [ ]:
X_users_log = X_users.apply(lambda x: np.where(x > 0, np.log(x + 1), 0))

X_users_scaled = pd.DataFrame(
    MinMaxScaler().fit_transform(X_users_log),
    columns=X_users.columns,
    index=X_users_log.index,
)

In [ ]:
X_users_scaled["score"] = X_users_scaled.apply(
    lambda x: (x.mean_loan_amount * 0.5)
    - (0.8 * x.mean_worst_delinquent_payments)
    - (0.8 * x.mean_infractor)
    + (0.1 * x.monthly_net_income)
    + (0.3 * x.count_loan_amount),
    axis=1,
)

In [ ]:
best_users_scoring = X_users_scaled.sort_values("score", ascending=False)[
    :10
].index.tolist()

In [ ]:
pd.merge(X_users, X_users_scaled[["score"]], left_index=True, right_index=True).loc[
    best_users_scoring
]

Podemos ver que tiene mucho sentido el score, son usuarios que nunca defaultearon, tomaron creditos grandes y los repagaron.

### WOE model

In [ ]:
def build_woe_features(dataset, variable_names, target_name, n_bins=10):
    data = dataset.copy()
    woe_features = {}
    for variable_name in variable_names:
        binned_feature, bins_ = pd.qcut(
            data[variable_name], n_bins, retbins=True, duplicates="drop"
        )
        binned_variable_name = f"{variable_name}_bins"
        data[binned_variable_name] = binned_feature
        tmp = data.groupby(binned_variable_name, as_index=False).agg(
            {target_name: ["count", "sum"]}
        )
        tmp.columns = [f"{variable_name}_bins", "N", "positive"]
        tmp["positive_rate"] = tmp.positive / tmp.positive.sum()
        tmp["negative"] = tmp.N - tmp.positive
        tmp["negative_rate"] = tmp.negative / tmp.negative.sum()
        tmp[f"WOE_{variable_name}"] = np.log(tmp.positive_rate / tmp.negative_rate)
        tmp[f"IV_{variable_name}"] = (tmp.positive_rate - tmp.negative_rate) * tmp[
            f"WOE_{variable_name}"
        ]
        woe_features[variable_name] = (
            tmp[
                [f"{variable_name}_bins", f"WOE_{variable_name}", f"IV_{variable_name}"]
            ],
            bins_,
        )
    return woe_features

In [ ]:
features = users_features.columns[1:]
target = "is_good"
woe_features = build_woe_features(users_features, features, target)

In [ ]:
def transform_df(data, woe_dfs, features, target):
    df = data.copy()
    for feature in features:
        woe_df, bins = woe_dfs.get(feature)
        if woe_df[f"IV_{feature}"].sum() < 0.15:
            logging.warning(f"{feature} discarded -> IV < 0.15")
            continue
        df[f"{feature}_bins"] = pd.cut(df[feature], bins, include_lowest=True)
        df = pd.merge(df, woe_df, on=f"{feature}_bins", how="left")
    cols = [column for column in df.columns if column.startswith("WOE")] + [target]
    df = df[cols]
    return df

In [ ]:
transformed_df = transform_df(users_features, woe_features, features, target)

In [ ]:
transformed_df

In [ ]:
X = transformed_df.drop(target, axis=1)
y = transformed_df[target]

In [ ]:
y.value_counts()

El dataset esta balanceado

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
import statsmodels.api as sm

# building the model and fitting the data
log_reg = sm.Logit(y_train, X_train).fit()
print(log_reg.summary())

Podemos ver que ni WOE_mean_infractor ni WOE_mean_payment_frequency days son significativas, veamos que tan bien predice el modelo

In [ ]:
reg = LogisticRegression()

reg.fit(X_train, y_train)

In [ ]:
plot_confusion_matrix(reg, X_test, y_test, cmap="Blues")

In [ ]:
print(classification_report(y_test, reg.predict(X_test)))

In [ ]:
plot_roc_curve(reg, X_test, y_test)

In [ ]:
check_users = X_test.copy()
check_users["real"] = y_test
check_users["prob"] = reg.predict_proba(X_test)[:, 1]

In [ ]:
plt.figure(figsize=(12, 8))
sns.distplot(check_users.query("real == 0")["prob"], label="negative", bins=25)
sns.distplot(check_users.query("real == 1")["prob"], label="positive", bins=25)
plt.legend()

Tiene una performance razonable aunque la distincion de clases no pareceria estar muy determinada

Ahora construyamos el scorecard a partir del modelo entrenado, recordemos que la formula para cada feature era:

$$ Score_i= (\beta_i * WoE_i + \frac{\alpha}{n}) * Factor + \frac{Offset}{n} $$

Donde:
- $\beta$ -> coeficiente de la regresion
- $\alpha$ -> el intercept de la regresion
- WoE -> Weight of Evidence de la feature
- n -> la cantidad de features del modelo
- Factor -> pdo (points to double the odds) /Ln(2) 
- Offset = Score - (Factor × ln(Odds))

In [ ]:
n = len(X)
pdo = 20
factor = pdo / np.log(2)
offset = 200
# import ipdb; ipdb.set_trace()
coefs_dict = dict(zip(X, reg.coef_[0]))
coefs_dict["intercept"] = reg.intercept_[0]
results = transformed_df.copy()
for feature in X.columns.tolist():
    feature_ = "_".join(feature.split("_")[1:])
    df = woe_features.get(feature_, ())[0]
    coef = coefs_dict.get(feature)
    intercept = coefs_dict.get("intercept")
    df["score"] = df[f"WOE_{feature_}"].apply(
        lambda x: (x * coef + (intercept / n) * factor + (offset / n))
    )
    results = pd.merge(
        results,
        woe_features.get(feature_)[0][[f"WOE_{feature_}", "score"]],
        on=f"WOE_{feature_}",
    )

In [ ]:
score_cols = [col for col in results.columns if col.startswith("score")]
score = results[score_cols].sum(axis=1)

In [ ]:
check = X_users.copy()

In [ ]:
check["score"] = score

In [ ]:
check.sort_values("score", ascending=False)[:10]

In [ ]:
best_users_woe = check.sort_values("score", ascending=False)[:10].index.tolist()

### Infractor model

In [ ]:
infractor_features["infractor"].value_counts()

Las etiquetas estan muy desbalanceadas asi que hay 2 posibilidades:
- Samplear algunas observaciones negativas para que queden mejor distribuidas
- Sobresamplear utilizando tecnicas como [SMOTE](https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/)

Dado que estamos utilizando features que tienen dependencia temporal me surgio la duda si era correcto sobresamplear asi que opte por subsamplear

In [ ]:
neg = infractor_features.query("infractor == 0").sample(800)
pos = infractor_features.query("infractor == 1")
infractor_features_subsample = neg.append(pos).sort_index()  # Avoid shuffling data

In [ ]:
infractor_features_subsample["infractor"].value_counts()

In [ ]:
train_size = 0.75
n_obs = int(train_size * infractor_features_subsample.shape[0])
train_infractor = infractor_features_subsample[:n_obs]
test_infractor = infractor_features_subsample[n_obs + 1 :]
X_train_infractor = train_infractor.drop(["infractor", "user_id"], axis=1)
y_train_infractor = train_infractor["infractor"]
X_test_infractor = test_infractor.drop(["infractor", "user_id"], axis=1)
y_test_infractor = test_infractor["infractor"]

In [ ]:
def build_pipeline(numeric_features, categorical_features, model):
    numeric_transformer = Pipeline(
        [
            (
                "imputer",
                SimpleImputer(strategy="most_frequent"),
            ),
        ]
    )
    categorical_transformer = Pipeline(
        [
            ("imputer", SimpleImputer(strategy="constant", fill_value="__")),
            ("onehot", OneHotEncoder(handle_unknown="ignore")),
        ]
    )
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features),
        ]
    )

    pipe = Pipeline([("preprocessor", preprocessor), ("estimator", model)])
    return pipe

In [ ]:
categorical_features = [
    "institution",
    "payment_frequency_days",
    "loan_created_year",
    "loan_created_month",
    "loan_created_weekday",
    "loan_expected_end_year",
    "loan_expected_end_month",
    "loan_expected_end_weekday",
]
numerical_features = X_train_infractor.drop(
    categorical_features, axis=1
).columns.tolist()
# model = GridSearchCV(LGBMClassifier(), params__, cv=5, n_jobs=-1)
pipe = build_pipeline(numerical_features, categorical_features, LGBMClassifier())

In [ ]:
pipe.fit(X_train_infractor, y_train_infractor)

In [ ]:
check_infractor = X_test_infractor.copy()
check_infractor["preds"] = pipe.predict(X_test_infractor)
check_infractor["prob"] = pipe.predict_proba(X_test_infractor)[:, 1]
check_infractor["real"] = y_test_infractor

In [ ]:
plot_confusion_matrix(pipe, X_test_infractor, y_test_infractor, cmap="Blues")

In [ ]:
print(classification_report(check_infractor["real"], check_infractor["preds"]))

In [ ]:
plt.figure(figsize=(12, 8))
sns.distplot(check_infractor.query("real == 0")["prob"], label="negative", bins=25)
sns.distplot(check_infractor.query("real == 1")["prob"], label="positive", bins=25)
plt.legend()

In [ ]:
def get_pipeline_feature_importance(numeric_features, categorical_features, pipeline):
    ohe_feature_names = (
        pipeline.steps[0][1]
        .transformers_[1][1]
        .steps[1][1]
        .get_feature_names(categorical_features)
    )
    feature_importances = pipeline.steps[1][1].feature_importances_
    cols = list(numeric_features) + list(ohe_feature_names)
    f_i = list(zip(cols, feature_importances))
    feat_imp = pd.DataFrame(f_i, columns=["feature_names", "importance"])
    feat_imp = feat_imp.sort_values("importance", ascending=False).reset_index(
        drop=True
    )
    return feat_imp

In [ ]:
get_pipeline_feature_importance(numerical_features, categorical_features, pipe)[:5]

## 5) Model Deployment

En esta seccion vamos a construir las features que representarian como funcionaria nuestra logica en produccion y determinar, segun los usuarios que elegimos en el punto anterior, que monto, que tasa y a que plazo le ofreceriamos un prestamo. Dado que construimos dos modelos para seleccionar usuarios, vamos a probarlo para ambos casos

In [ ]:
user_features_woe = (
    infractor_features.groupby("user_id")
    .max()[
        [
            "monthly_income",
            "monthly_outcome",
            "loan_amount",
            "historic_total_loans_granted",
            "historic_worst_delinquent_payments",
            "historic_infractor_rate",
            "historic_pre_cancelled_rate",
        ]
    ]
    .reset_index()
    .query(f"user_id in {best_users_woe}")
)

In [ ]:
user_features_scoring = (
    infractor_features.groupby("user_id")
    .max()[
        [
            "monthly_income",
            "monthly_outcome",
            "loan_amount",
            "historic_total_loans_granted",
            "historic_worst_delinquent_payments",
            "historic_infractor_rate",
            "historic_pre_cancelled_rate",
        ]
    ]
    .reset_index()
    .query(f"user_id in {best_users_scoring}")
)

In [ ]:
user_features_scoring.columns

Ahora construimos las features base asociadas a la institucion y el tipo de credito que se otorga para probar con distintos tipos y quedarnos con la mejor combinacion
- Institucion que otorga el credito: Konfio
- La cantidad que se le va a ofrecer: maxima cantidad prestada y completada historica
- Duracion del credito: vamos a probar con 1, 2 y 3 años
- Frecuencia de pago: vamos a probar con todas las vistas anteriormente (semanal, quincenal, mensual, anual, etc)

In [ ]:
data = []
today = date.today()
for payment_frequency in [30, 7, 15, 14, 90, 360, 60]:
    for duration_days in [360, 360 * 2, 360 * 3]:
        data.append(
            dict(
                institution="KONFIO",
                payment_frequency_days=payment_frequency,
                duration_days=duration_days,
                total_credit_payments=duration_days / payment_frequency,
                loan_created_year=today.year,
                loan_created_month=today.month,
                loan_created_weekday=today.weekday(),
                loan_expected_end_year=(today + timedelta(days=duration_days)).year,
                loan_expected_end_month=(today + timedelta(days=duration_days)).month,
                loan_expected_end_weekday=(
                    today + timedelta(days=duration_days)
                ).weekday(),
            )
        )
base_data_df = pd.DataFrame(data)

In [ ]:
base_data_df.columns

Con esto ya tenemos todas las features necesarias para evaluar en nuestro modelo. Generemos el dataset con todas las combinacion de frequencia de pago + duracion del credito que pusimos anteriormente para los usuarios elegidos por el modelo de scoring

In [ ]:
base_data_records = base_data_df.to_dict(orient="records")
user_records_scoring = user_features_scoring.drop("user_id", axis=1).to_dict(
    orient="records"
)
user_records_woe = user_features_woe.drop("user_id", axis=1).to_dict(orient="records")

observations_scoring = pd.DataFrame(
    [
        {**base_data_record, **user_record}
        for base_data_record in base_data_records
        for user_record in user_records_scoring
    ]
)
observations_woe = pd.DataFrame(
    [
        {**base_data_record, **user_record}
        for base_data_record in base_data_records
        for user_record in user_records_woe
    ]
)
observations_scoring = observations_scoring[X_infractor.columns]
observations_woe = observations_woe[X_infractor.columns]

Predecimos sobre el dataset

In [ ]:
observations_check_scoring = observations_scoring.copy()
observations_check_scoring["preds"] = pipe.predict_proba(observations_scoring)[:, 0]

observations_check_woe = observations_woe.copy()
observations_check_woe["preds"] = pipe.predict_proba(observations_woe)[:, 0]

Y nos quedamos con la combinacion que devuelva la MAYOR probabilidad de NO ser un infractor

In [ ]:
best_combinations_woe = (
    observations_check_woe.groupby(["monthly_income", "monthly_outcome"])
    .idxmax()["preds"]
    .values.tolist()
)
best_combinations_scoring = (
    observations_check_scoring.groupby(["monthly_income", "monthly_outcome"])
    .idxmax()["preds"]
    .values.tolist()
)

In [ ]:
loan_givers_woe = observations_check_woe.loc[best_combinations_woe]
loan_givers_scoring = observations_check_scoring.loc[best_combinations_scoring]

In [ ]:
loan_givers_woe["anual_interest_rate"] = 0.25 * (1 / loan_givers_woe["preds"])
loan_givers_scoring["anual_interest_rate"] = 0.25 * (1 / loan_givers_scoring["preds"])

In [ ]:
loan_givers_woe

In [ ]:
loan_givers_scoring

Es llamativo el hecho de que el modelo prediga que van a ser infractores algunos de los usuarios seleccionados previamente (para ambas metodologias) me quedaria tratar de entender de donde puede surgir dicha discrepancia.

## Conclusion

Definimos 2 metodologias para elegir "mejores usuarios" y un modelo para medir la probabilidad de incumplir los pagos de un credito y en base a eso calcular una tasa de interes anual que se ajuste a dicha probabilidad. Ambos modelos tienen performance razonables, quedaria ver si la logica es extensible a otros tipos de creditos y no solo prestamos personales

## **< Anexo >**

In [ ]:
tmp = credit_reports.query(
    """ 
    amount_to_pay_next_payment != 0 and \
    current_balance != 0 and \
    total_credit_payments > 0 and \
    credit_type == 'Préstamo Personal ' and \
    worst_delinquency == 0 and \
    number_of_payments_due == 0 and \
    past_due_balance == 0
    """
)

In [ ]:
tmp["interest_rate"] = tmp.apply(
    lambda x: (
        (x.amount_to_pay_next_payment * x.total_credit_payments)
        / x.maximum_credit_amount
    )
    - 1,
    axis=1,
)

In [ ]:
sns.distplot(tmp.query("institution == 'KONFIO'")["interest_rate"])

In [ ]:
tmp["interest_rate"].describe()

En general, observo tasas razonables pero tambien veo casos que me hacen dudar si la logica que estoy siguiendo es correcta, asi que voy a dejar de profundizar sobre este aspecto porque no voy a utilizar dicha informacion